In [ ]:




# TODO pull in the asset universe from inclusion crit
# TODO see if i want the asset info on the assets for anything useful; otherwise ignore
# -pull asset metadate to get pow, pos, creation date, and maybe anything else tha tis useful
# TODO see if anything useful about exchanges; maybe get total volume if they have somewhere
# -remove the function from the OG script?

# TODO pull over needed functions as a copy

# TODO pull all the markets with the pullMarketInfo function; adjust to work for both freq; specificied assets as well
# TODO pull USDT and USDC exchange rates; adjust to work for both freq
# TODO pull market data function but need to adjust to work either for daily level or hourly level;  specificied assets as well
# -pull all markets in those three stables for all my assets to get prices, volume, and trades at HOURLY freq
# --so check first if the market offers hourly freq?
# TODO pull all available asset metrics at given freq for assets in universe
# -figure out all possible metrics to cut out ones that i am not interested in
# TODO PUlL BEST BID AND ASK FOR ALL THOSE MARKETS AT HOURLY FREQ 

# TODO move functions to the helper function call

# TODO adjust the OG script to see if they work for it too


# TODO FOR TARGET EXCHANGES AS WELL AS A FEW OTHER MAJORS pull all the available exchange metrics for my study period
# -try to get inflows and outflows?
# -pull open interest in futures? maybe as macro variable but lets see if i can get for a bunch of assets
# TODO: pull defi balance sheets
# TODO: see if i am missing anything as this is really great op for data
# TODO: taxonomy?



